In [84]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [12]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/Users/yazhinigopal/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [13]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
html = browser.html

True

In [14]:
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [15]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'7 Things to Know About the Mars 2020 Perseverance Rover Mission'

In [16]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission."

### Featured Image

In [60]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)
browser.find_by_tag('button')[1].click()

In [64]:
html = browser.html
img_soup = soup(html,'html.parser')
img_url = img_soup.find('img',class_='fancybox-image')['src']
img_url = url + img_url
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Facts

In [65]:
url = 'https://galaxyfacts-mars.com/'
browser.visit(url)
table_soup = soup(browser.html,'html.parser')

In [169]:
# Conventional Scraping
facts_table = table_soup.table.tbody.find_all('tr')
facts = {}
for i in facts_table:
    desc = i.th.text.strip(' :')
    mars = i.find_all('td')[0].text.strip()
    earth = i.find_all('td')[1].text.strip()
    facts[desc] = [mars,earth]
facts_df = pd.DataFrame.from_dict(facts,orient='index')
facts_df.index.set_names(facts_df.index[0], inplace=True)
facts_df.columns = [facts_df.iloc[0,0],facts_df.iloc[0,1]]
facts_df = facts_df.iloc[1:]
facts_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter,"6,779 km","12,742 km"
Mass,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons,2,1
Distance from Sun,"227,943,824 km","149,598,262 km"
Length of Year,687 Earth days,365.24 days
Temperature,-87 to -5 °C,-88 to 58°C


In [172]:
# Pandas Scraping
df = pd.read_html(url)[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df = df.iloc[1:]
df

,Mars,Earth
description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [173]:
browser.quit()